### COMP - 472
Assignment Two

---------

#### Task 1

1) Add imports

In [1]:
import os
import csv
import random
import pandas as pd
import gensim.downloader as api
from gensim.models import KeyedVectors

2) Load model

In [2]:
model_name = 'word2vec-google-news-300'
model = KeyedVectors.load_word2vec_format(api.load(model_name, return_path=True), binary=True) 

3) Load test dataset

In [3]:
dataset_path = 'A2-DataSet/synonym.csv'
df = pd.read_csv(dataset_path)

4) Run the model on test data and report result

In [4]:
details_output_file = os.path.join('output', f'{model_name}-details.csv')
analysis_output_file = os.path.join('output', f'analysis.csv')

correct_labels = 0
answered_questions = 0

In [5]:
with open(details_output_file, mode='w', newline='') as details_file:
  details_writer = csv.writer(details_file)
  details_writer.writerow(['question-word', 'correct-answer', 'guess-word', 'label'])

  for _, row in df.iterrows():
    question_word, correct_answer, guess_words = row['question'], row['answer'], row[2:].to_list()
    guess_words_in_vocab = [word for word in guess_words if word in model.index_to_key]

    if question_word in model.index_to_key and len(guess_words_in_vocab) > 0:
      most_similar_word = model.most_similar_to_given(question_word, guess_words_in_vocab)
      label = 'correct' if most_similar_word == correct_answer else 'wrong'
      correct_labels += 1 if label == 'correct' else 0
      answered_questions += 1
    else:
      label = 'guess'
      most_similar_word = random.choice(guess_words)

    details_writer.writerow([question_word, correct_answer, most_similar_word, label])

In [13]:
accuracy = correct_labels / answered_questions if answered_questions > 0 else 0

with open(analysis_output_file, mode='a+', newline='') as analysis_file:
  analysis_writer = csv.writer(analysis_file)
  analysis_writer.writerow(['Model Name', 'Vocab Length', 'Number of Correct Labels', 'Number of Answered Questions', 'Accuracy of Answered Questions'])
  analysis_writer.writerow([model_name, len(model.index_to_key), correct_labels, answered_questions, accuracy])

------------
#### Task 2

In [ ]:
# TODO: Add code for task 2 here

------------
#### Task 3

In [ ]:
# TODO: Add code for task 3 here